In [88]:
#encoding=utf-8
import csv
import itertools
import operator
import numpy as np
import sys
import matplotlib.pyplot as plt
from datetime import datetime
from utils import *

%matplotlib inline

In [16]:
climates = []
infile = open("../data/climate_hangzhou.csv","rb")
jump_header = False
for row in infile:
    if not jump_header:
        jump_header = True
        continue
    row = row.strip().decode("utf-8")
    items = row.split(",")
    dates = items[0]
    rainfall_morning = float(items[1])
    rainfall_afternoon = float(items[2])
    rainfall_night = float(items[3])
    min_temperature = float(items[4])
    max_temperature = float(items[5])
    wind_forece = float(items[6])
    climates.append([rainfall_afternoon,min_temperature,max_temperature,wind_forece])

In [101]:
X_train = np.asarray([day_climate for day_climate in climates[:-1]])
Y_train = np.asarray([day_climate for day_climate in climates[1:]])

In [65]:
class RNNNumpy:
    def __init__(self, input_dim, hidden_dim=3, bptt_truncate=4):
        # Assign instance variables
        self.input_dim = input_dim
        self.output_dim = input_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./input_dim), np.sqrt(1./input_dim), (hidden_dim, input_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (input_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))

In [104]:
def forward_propagation(self, x):
    # The total number of time steps
    T = len(x)
    # During forward propagation we save all hidden states in s because need them later.
    # We add one additional element for the initial hidden, which we set to 0
    s = np.zeros((T + 1, self.hidden_dim))
    s[-1] = np.zeros(self.hidden_dim)
    # The outputs at each time step. Again, we save them for later.
    o = np.zeros((T, self.output_dim))
    # For each time step...
    for t in np.arange(T):
        s[t] = np.tanh(self.U.dot(x[t]) + self.W.dot(s[t-1]))
        o[t] = softmax(self.V.dot(s[t]))
    return [o, s]
 
RNNNumpy.forward_propagation = forward_propagation

In [115]:
#square error
def calculate_total_loss(self, x, y):
    L = 0
    o, s = self.forward_propagation(x)
    for i in range(len(y)):
        L += 0.5 * np.sum(np.square(np.array(y[i])-np.array(o[i])))
    return L

#mean square error
def calculate_loss(self, x, y):
    # Divide the total loss by the number of training examples
    N = np.sum((len(y_i) for y_i in y))
    return self.calculate_total_loss(x,y)/N

RNNNumpy.calculate_total_loss = calculate_total_loss
RNNNumpy.calculate_loss = calculate_loss

In [116]:
model = RNNNumpy(4)
print "mse: %f" % model.calculate_loss(X_train[:1000], Y_train[:1000])

mse: 110.541113


In [ ]:
def bptt(self, x, y):
    T = len(y)
    # Perform forward propagation
    o, s = self.forward_propagation(x)
    # We accumulate the gradients in these variables
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    
    # For each output backwards...
    for t in np.arange(T)[::-1]:
        dLdV += np.outer(delta_o[t], s[t].T)
        # Initial delta calculation
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        # Backpropagation through time (for at most self.bptt_truncate steps)
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
            # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t
            # Update delta for next step
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdU, dLdV, dLdW]
 
RNNNumpy.bptt = bptt